In [32]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [33]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [34]:
train['TotalLivingSF'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalLivingSF'] = test['BsmtFinSF1'] + test['BsmtFinSF2'] + test['1stFlrSF'] + test['2ndFlrSF']

train['TotalLivingSF'] = np.log(train['TotalLivingSF'])
test['TotalLivingSF'] = np.log(test['TotalLivingSF'])

In [35]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
TotalLivingSF    0.698827
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr

In [36]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold', 'TotalLivingSF']


In [37]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition']

# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition']


In [38]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(0, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(0, inplace=True)

In [39]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()

train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])

train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBsmt'] = 1
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train['1stFlrSF'] = np.log(train['1stFlrSF'])
test['1stFlrSF'] = np.log(test['1stFlrSF'])

train['HasBsmtFin'] = pd.Series(len(train['BsmtFinSF1']), index=train.index)
train['HasBsmtFin'] = 0 
train.loc[train['BsmtFinSF1']>0,'HasBsmtFin'] = 1
train.loc[train['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(train['BsmtFinSF1'])
test['HasBsmtFin'] = pd.Series(len(test['BsmtFinSF1']), index=test.index)
test['HasBsmtFin'] = 0 
test.loc[test['BsmtFinSF1']>0,'HasBsmtFin'] = 1
test.loc[test['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(test['BsmtFinSF1'])

train['HasWoodDeck'] = pd.Series(len(train['WoodDeckSF']), index=train.index)
train['HasWoodDeck'] = 0 
train.loc[train['WoodDeckSF']>0,'HasWoodDeck'] = 1
train.loc[train['HasWoodDeck']==1,'WoodDeckSF'] = np.log(train['WoodDeckSF'])
test['HasWoodDeck'] = pd.Series(len(test['WoodDeckSF']), index=test.index)
test['HasWoodDeck'] = 0 
test.loc[test['WoodDeckSF']>0,'HasWoodDeck'] = 1
test.loc[test['HasWoodDeck']==1,'WoodDeckSF'] = np.log(test['WoodDeckSF'])


/home/marek/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [40]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
TotalLivingSF    0.741062
GrLivArea        0.730255
GarageCars       0.680625
GarageArea       0.650888
1stFlrSF         0.608947
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.534422
Fireplaces       0.489449
MasVnrArea       0.426775
TotalBsmtSF      0.373009
GarageYrBlt      0.349014
WoodDeckSF       0.343269
OpenPorchSF      0.321053
HasWoodDeck      0.320349
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209044
BsmtFinSF1       0.207962
HasBsmt          0.199634
LotFrontage      0.179303
HasBsmtFin       0.162231
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057329
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF

In [41]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 47 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-nu

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [43]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 29)
(1459, 29)
(1460, 137)
(1459, 137)


In [44]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=350, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)

forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.0014825300524898655

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

model = Lasso(alpha=0.0001, max_iter=1e9)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, train_full_prepared, train_y, 
                         scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

lasso_rmse_scores = np.sqrt(-scores)
display_scores(lasso_rmse_scores)

print(pd.Series(lasso_rmse_scores).describe())

TypeError: __init__() got an unexpected keyword argument 'shuffle'

In [48]:
from numpy import arange
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
# define model
model = Lasso(max_iter=1e9)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, verbose=3)
# perform the search
results = search.fit(train_full_prepared, train_y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)


Fitting 30 folds for each of 8 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    2.1s


MAE: -0.017
Config: {'alpha': 0.0003}


[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    2.5s finished


In [49]:
cvres = results.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.13084740997825572 {'alpha': 0.0001}
0.1300696516596726 {'alpha': 0.0002}
0.12982972405774004 {'alpha': 0.0003}
0.12984322552269567 {'alpha': 0.0004}
0.12995351256420162 {'alpha': 0.0005}
0.13015252541452127 {'alpha': 0.0006}
0.13047055263420176 {'alpha': 0.0007}
0.1308445601965973 {'alpha': 0.0008}


In [54]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(results.best_estimator_.feature_importances_, attributes), reverse=True)

AttributeError: 'Lasso' object has no attribute 'feature_importances_'

In [14]:
print(housing_predictions[0])
print(train_y[0])

12.247694320220951
12.247694320220994


In [15]:
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

207.04718512192318

In [16]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
208499.9999999911


In [17]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(forest_reg.feature_importances_, attributes), reverse=True)

[(0.18524400183922088, 'OverallQual'),
 (0.1283418815904073, 'TotalLivingSF'),
 (0.12044759171456802, 'GrLivArea'),
 (0.08776379660423454, 'YearBuilt'),
 (0.04835838461728762, 'GarageCars'),
 (0.04731600063596822, 'TA'),
 (0.0344084548321465, 'TotalBsmtSF'),
 (0.03425052502258137, '1stFlrSF'),
 (0.03279461980205903, 'GarageArea'),
 (0.03258363992198943, 'FullBath'),
 (0.03013116825415528, 'GarageYrBlt'),
 (0.0196448548785311, 'LotArea'),
 (0.01688100014256175, 'YearRemodAdd'),
 (0.01670921570559378, 'Fireplaces'),
 (0.01391786987851443, 'BsmtFinSF1'),
 (0.011869877962659264, '2ndFlrSF'),
 (0.010081403913480362, 'OverallCond'),
 (0.009324404689334988, 'N'),
 (0.008703744377841946, 'Y'),
 (0.007710538297088906, 'BsmtUnfSF'),
 (0.0070742135783931764, 'TotRmsAbvGrd'),
 (0.007006491907925024, 'PConc'),
 (0.005897755725616981, 'OpenPorchSF'),
 (0.005368119251799637, 'LotFrontage'),
 (0.0049244703221955075, 'Gd'),
 (0.004689921991407094, 'MasVnrArea'),
 (0.0045419189767751346, 'MSSubClass'),


In [25]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=450, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

TypeError: cross_val_score() got an unexpected keyword argument 'shuffle'

In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
    {'n_estimators': [100, 200, 300, 350, 400, 450], 'max_features': [16, 32, 64]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [100, 200, 300, 350, 400, 450], 
     'max_features': [16, 32, 64]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=3, n_jobs=-1)
grid_search.fit(train_full_prepared, train_y)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  3.0min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [16, 32, 64],
   

In [21]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 32, 'n_estimators': 450}

In [22]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=32, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=450,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [23]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.13819125682937164 {'max_features': 16, 'n_estimators': 100}
0.13738795238586812 {'max_features': 16, 'n_estimators': 200}
0.13682479364639416 {'max_features': 16, 'n_estimators': 300}
0.13668269904258343 {'max_features': 16, 'n_estimators': 350}
0.13691310388920702 {'max_features': 16, 'n_estimators': 400}
0.1368491914316079 {'max_features': 16, 'n_estimators': 450}
0.13785764443382584 {'max_features': 32, 'n_estimators': 100}
0.1368810905213422 {'max_features': 32, 'n_estimators': 200}
0.13588687453908863 {'max_features': 32, 'n_estimators': 300}
0.13587249837353724 {'max_features': 32, 'n_estimators': 350}
0.13587721592976412 {'max_features': 32, 'n_estimators': 400}
0.13599638056763205 {'max_features': 32, 'n_estimators': 450}
0.13678998686726665 {'max_features': 64, 'n_estimators': 100}
0.13667053531472315 {'max_features': 64, 'n_estimators': 200}
0.13681513615963795 {'max_features': 64, 'n_estimators': 300}
0.1368119212841651 {'max_features': 64, 'n_estimators': 350}
0.136842290

In [24]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([4.45126665e-03, 9.18402277e-03, 1.81381503e-02, 1.90648143e-01,
       1.04727165e-02, 8.71248937e-02, 1.59410953e-02, 4.63486810e-03,
       1.28499120e-02, 7.67928301e-03, 3.77892984e-02, 3.35876587e-02,
       1.18368531e-02, 1.19554835e-01, 1.49146890e-03, 3.18946464e-02,
       2.47763466e-03, 3.69653132e-03, 2.29343903e-03, 6.61763054e-03,
       1.59648497e-02, 2.58574216e-02, 4.64894528e-02, 3.96132177e-02,
       3.03009798e-03, 6.05908572e-03, 2.04783027e-03, 2.30929407e-03,
       1.23764446e-01, 2.80970391e-05, 6.00723566e-06, 5.68488190e-05,
       2.87366499e-04, 3.66390265e-04, 2.74859353e-04, 1.65767594e-03,
       1.44306853e-03, 1.96246898e-04, 7.39075745e-04, 2.99043156e-04,
       1.79706154e-04, 7.20167869e-04, 2.53090209e-05, 2.52869314e-04,
       2.04683851e-04, 2.80739089e-04, 1.31096756e-03, 1.40629511e-04,
       4.72544276e-04, 1.55541937e-04, 2.91481502e-04, 2.05793858e-04,
       1.17556223e-04, 5.54089447e-05, 1.21223081e-03, 1.02889063e-04,
      

In [25]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.1906481429797765, 'OverallQual'),
 (0.12376444557572433, 'TotalLivingSF'),
 (0.1195548346212969, 'GrLivArea'),
 (0.08712489369115749, 'YearBuilt'),
 (0.04852685168472486, 'TA'),
 (0.046489452762279754, 'GarageCars'),
 (0.039613217729576676, 'GarageArea'),
 (0.037789298383509126, 'TotalBsmtSF'),
 (0.033587658709761596, '1stFlrSF'),
 (0.031894646444014255, 'FullBath'),
 (0.02585742156337479, 'GarageYrBlt'),
 (0.01813815034577932, 'LotArea'),
 (0.01596484971544268, 'Fireplaces'),
 (0.015941095345196443, 'YearRemodAdd'),
 (0.012849911964804046, 'BsmtFinSF1'),
 (0.01183685306828614, '2ndFlrSF'),
 (0.010472716508927615, 'OverallCond'),
 (0.00927236887246009, 'Y'),
 (0.00918402277117282, 'LotFrontage'),
 (0.008249182876605964, 'N'),
 (0.007679283010564685, 'BsmtUnfSF'),
 (0.006736455983417287, 'PConc'),
 (0.0066176305437349375, 'TotRmsAbvGrd'),
 (0.006059085718137152, 'OpenPorchSF'),
 (0.004634868102684757, 'MasVnrArea'),
 (0.004451266654215717, 'MSSubClass'),
 (0.004304700421924606, 'Gd'

In [50]:
# y_pred = forest_reg.predict(test_full_prepared)
# y_pred = grid_search.predict(test_full_prepared)
y_pred = results.predict(test_full_prepared)


y_pred_normal = np.e**y_pred
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [51]:
sub.to_csv("data/submission_script34_lasso.csv", index=False)

In [28]:
sub.shape

(1459, 2)